In [2]:
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt


In [3]:
mongo = MongoClient(port=27017)

In [4]:
print(mongo.list_database_names())

['Spotify_2023', 'admin', 'autosaurus', 'classDB', 'config', 'local', 'met', 'uk_food']


In [7]:
db = mongo['Spotify_2023']


In [9]:
print(db.list_collection_names())

['Song_list']


In [10]:
Most_Popular_Songs_2023 = db['Song_list']

In [12]:
Most_Popular_Songs_2023.find_one()

{'_id': ObjectId('666b8730480d52d219eeeabd'),
 'option': 'Seven (feat. Latto) (Explicit Ver.)',
 'artist(s)_name': 'Latto, Jung Kook',
 'artist_count': 2,
 'released_year': 2023,
 'released_month': 7,
 'released_day': 14,
 'in_spotify_playlists': 553,
 'in_spotify_charts': 147,
 'streams': 141381703,
 'in_apple_playlists': 43,
 'in_apple_charts': 263,
 'in_deezer_playlists': 45,
 'in_deezer_charts': 10,
 'in_shazam_charts': 826,
 'bpm': 125,
 'key': 'B',
 'mode': 'Major',
 'danceability_%': 80,
 'valence_%': 89,
 'energy_%': 83,
 'acousticness_%': 31,
 'instrumentalness_%': 0,
 'liveness_%': 8,
 'speechiness_%': 4}